## Задание на кластеризацию. Срок: 5 декабря.

In [ ]:
import numpy as np
import pandas as pd
import random
import math
import sklearn.datasets as ds
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

In [ ]:
# Загрузите данные по покимонам

X = pd.read_csv("../data/Pokemon.csv", index_col=0)
X.shape

In [ ]:
y = list(X['Legendary'].map({True: 1, False:0}))
X = X._get_numeric_data().dropna(axis=1).drop('Legendary', axis=1)

In [ ]:
# Спойлер: попробуйте решить задачу со скейлингом и без

# ms = MinMaxScaler()
# ms.fit(X)
# X = ms.transform(X)

In [ ]:
# С помощью TSNE отобразим данные на плоскости 
from sklearn.manifold import TSNE
tsne = TSNE(random_state=17, verbose=2, n_iter=10000, n_iter_without_progress=1000)

res = tsne.fit_transform(X)

In [ ]:
# Посмотрим на реальное расположение классов

plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=y, s=10)
plt.title('TSNE')

Видно, что классы сгруппировались в одном месте. Но там не только они. Интересно, наши алгоритмы смогут что-то сделать?

### Kmeans

In [ ]:
Ks = range(1, 15)
score = [KMeans(n_clusters=i, n_jobs=-1).fit(X).inertia_ for i in Ks]

In [ ]:
plt.plot(Ks, score)

In [ ]:
# Можно попробовать 3 или 4 кластера

kmeans = KMeans(n_clusters=3, random_state=1)
kmeans.fit(X)

plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=kmeans.labels_, s=10, cmap='viridis')
plt.title('TSNE')

Как и казалось очевидным, kmean побил на 3 видимых класса. И да, он поместил все семплы с таргетом 1 в один класс. На самом деле, это может быть полезным.

#### Метрики

Помимо метрики Silhouette, которую мы рассматривали на семинаре, есть метрики, которые считают качество кластеризации на данных, где есть разметка. Вместо данных они на вход получают кластеры и реальную разметку.

Ниже перечислены некоторые из этих метрик. Как думаете, какая самая уместная в нашей задаче?

from sklearn import metrics

        'AMI': metrics.adjusted_mutual_info_score(y, preds),
        'ARI': metrics.adjusted_rand_score(y, preds),
        'V-measure': metrics.v_measure_score(y, preds),
        'Homogenity': metrics.homogeneity_score(y, preds),
        'Completeness': metrics.completeness_score(y, preds)

### DBSCAN

Посмотрим, как DBSCAN справится с задачей кластеризации. Переберем разные значения параметров алгоритма и сравним качество по метрикам.

Максимизация какой метрики, на ваш взгляд, дает лучшее разбиение?

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
eps = np.linspace(30, 130, 101)
# Если будете скалировать, то вам нужно будет взять eps < 1. Почему?
samples = range(3, 10)

scores = []
best_sc = -1
best_ep = None
best_sa = None
for ep in eps:
    for sa in samples:
        clst = DBSCAN(eps=ep, min_samples=sa)
        pred = clst.fit_predict(X)
        try:
            sc = metrics.adjusted_mutual_info_score(y, pred, average_method='geometric')
            if sc > best_sc:
                best_sc = sc
                best_ep = ep
                best_sa = sa
            scores.append((ep, sa, sc))
        except:
            pass

clst = DBSCAN(eps=best_ep, min_samples=best_sa)
pred = clst.fit_predict(X)
print(metrics.adjusted_mutual_info_score(y, pred, average_method='geometric'))

plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=pred, s=10, cmap='viridis')
plt.title('TSNE')

In [ ]:
eps = np.linspace(30, 130, 101)

samples = range(3, 10)

scores = []
best_sc = -1
best_ep = None
best_sa = None
for ep in eps:
    for sa in samples:
        clst = DBSCAN(eps=ep, min_samples=sa)
        pred = clst.fit_predict(X)
        try:
            sc = metrics.adjusted_rand_score(y, pred)
            if sc > best_sc:
                best_sc = sc
                best_ep = ep
                best_sa = sa
            scores.append((ep, sa, sc))
        except:
            pass

clst = DBSCAN(eps=best_ep, min_samples=best_sa)
pred = clst.fit_predict(X)
print(metrics.adjusted_rand_score(y, pred))

plt.figure(figsize=(12,8))
plt.scatter(res[:,0], res[:,1], c=pred, s=10, cmap='viridis')
plt.title('TSNE')

Повторите такую процедуру с остальными метриками и выберите, максимизация какой метрики, дает лучшее разбиение.

#### Еще кластеризация
Посмотрите на короткую справку по [типовым методам кластеризации](https://scikit-learn.org/stable/modules/clustering.html).

Реализуйте еще какой-нибудь алгоритм, который вам нравится, и проведите то же исследование, что было с DBSCAN.

Какой алгоритм и максимизация какой метрики дали наилучший визуальный результат? Почему?
